<a href="https://colab.research.google.com/github/thekirti02/Plant-disease/blob/main/ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 6.4 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=d787d63871076a4c22f6dc594bc625ce37a9e912b87047734bb9811b26c38821
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from skimage.feature import hog

# Define the path to your dataset
folder_path = '/content/drive/MyDrive/Plant_dataset'

# Lists to store preprocessed images and labels
images = []
labels = []

# Define larger image size for more detail
image_size = (64, 64)  # Adjust size to retain more information
resize_method = Image.BICUBIC

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Counters for healthy and rust images
healthy_count = 0
rust_count = 0

# Preprocess images and labels
for filename in os.listdir(folder_path):
    if filename.startswith('Healthy'):
        label = 'Healthy'
        healthy_count += 1
    elif filename.startswith('Rust'):
        label = 'Rust'
        rust_count += 1
    else:
        continue

    img = Image.open(os.path.join(folder_path, filename))
    img = img.resize(image_size, resample=resize_method)
    img = img.convert("L")  # Convert to grayscale for HOG feature extraction

    # Extract HOG features
    hog_features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)

    # Append the HOG features and label to respective lists
    images.append(hog_features)
    labels.append(label)

# Convert to arrays
X = np.array(images)
y = label_encoder.fit_transform(labels)

# Normalize the feature vectors
X = X / np.max(X)  # Normalize to range [0, 1]

# Split into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)



In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage.feature import hog

In [ ]:
#  Initialize classifiers
classifiers = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': SVC(kernel='linear', random_state=42),
    'Naive Bayes': GaussianNB(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    clf.fit(train_images, train_labels)
    predictions = clf.predict(test_images)
    accuracy = accuracy_score(test_labels, predictions)

    print(f"{name} Accuracy: {accuracy:.2f}")
    print(f"Classification Report for {name}:\n{classification_report(test_labels, predictions)}\n")

Logistic Regression Accuracy: 0.74
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.74      0.77      0.75       138
           1       0.74      0.71      0.73       131

    accuracy                           0.74       269
   macro avg       0.74      0.74      0.74       269
weighted avg       0.74      0.74      0.74       269


SVM Accuracy: 0.71
Classification Report for SVM:
              precision    recall  f1-score   support

           0       0.71      0.72      0.72       138
           1       0.71      0.69      0.70       131

    accuracy                           0.71       269
   macro avg       0.71      0.71      0.71       269
weighted avg       0.71      0.71      0.71       269


Naive Bayes Accuracy: 0.69
Classification Report for Naive Bayes:
              precision    recall  f1-score   support

           0       0.71      0.68      0.69       138
           1       0.68      0.70    

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier

# Define the path to your dataset
folder_path = '/content/drive/MyDrive/Plant_dataset'

# Lists to store preprocessed images and labels
images = []
labels = []

# Define image size and preprocessing steps
image_size = (4, 2)  # Adjust size as needed
resize_method = Image.BICUBIC

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Counters for healthy and rust images
healthy_count = 0
rust_count = 0

# Preprocess images and labels
for filename in os.listdir(folder_path):
    if filename.startswith('Healthy'):
        label = 'Healthy'
        healthy_count += 1
    elif filename.startswith('Rust'):
        label = 'Rust'
        rust_count += 1
    else:
        continue

    img = Image.open(os.path.join(folder_path, filename))
    img = img.resize(image_size, resample=resize_method)
    img_vector = np.array(img).flatten()[:8]  # Convert to vector and limit to 8 features

    # Append the image vector and label to respective lists
    images.append(img_vector)
    labels.append(label)

# Convert to arrays
X = np.array(images)
y = label_encoder.fit_transform(labels)

# Normalize images
X = X / 255.0  # Normalize pixel values to range [0, 1]

# Split into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=42)

# Use TPOTClassifier for AutoML
tpot = TPOTClassifier(verbosity=2, generations=10, population_size=50, random_state=42)


# Train the model
tpot.fit(train_images, train_labels)

# Evaluate the model
score = tpot.score(test_images, test_labels)
print(f"Accuracy: {score}")

# Export the best pipeline
tpot.export('best_pipeline.py')


Optimization Progress:   0%|          | 0/550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8306793650793651

Generation 2 - Current best internal CV score: 0.8322666666666667

Generation 3 - Current best internal CV score: 0.8338793650793651

Generation 4 - Current best internal CV score: 0.8338793650793651

Generation 5 - Current best internal CV score: 0.8338793650793651

Generation 6 - Current best internal CV score: 0.8338793650793651

Generation 7 - Current best internal CV score: 0.8386412698412699

Generation 8 - Current best internal CV score: 0.8386539682539682

Generation 9 - Current best internal CV score: 0.843415873015873

Generation 10 - Current best internal CV score: 0.843415873015873

Best pipeline: GradientBoostingClassifier(input_matrix, learning_rate=0.1, max_depth=8, max_features=0.05, min_samples_leaf=1, min_samples_split=6, n_estimators=100, subsample=0.8)
Accuracy: 0.8401486988847584


### Waste code

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tpot import TPOTClassifier

# Define the path to your dataset
folder_path = '/content/drive/MyDrive/Plant_dataset'

# Lists to store preprocessed images and labels
images = []
labels = []

# Define image size and preprocessing steps
image_size = (24, 24)  # Reduced size to lower memory usage
resize_method = Image.BICUBIC

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Preprocess images and labels
for filename in os.listdir(folder_path):
    if filename.startswith('Healthy'):
        label = 'Healthy'
    elif filename.startswith('Rust'):
        label = 'Rust'
    else:
        continue

    # Open and resize image
    img = Image.open(os.path.join(folder_path, filename))
    img = img.resize(image_size, resample=resize_method)

    # Convert the image to a vector (flatten)
    img_vector = np.array(img).flatten()  # Extract more features by flattening all pixels

    # Append the image vector and label to respective lists
    images.append(img_vector)
    labels.append(label)

# Convert to arrays
X = np.array(images)
y = label_encoder.fit_transform(labels)

# Normalize images
X = X / 255.0  # Normalize pixel values to range [0, 1]

# Split into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=42)



In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from skimage.transform import resize
from sklearn.decomposition import PCA
import numpy as np
from tpot import TPOTClassifier

# Step 1: Resize images for memory and computational efficiency
train_images_resized = np.array([resize(img, (256, 256)) for img in train_images])
test_images_resized = np.array([resize(img, (256, 256)) for img in test_images])

# Step 2: Flatten the images from (256, 256, 3) to (256*256*3,)
train_images_flat = train_images_resized.reshape(len(train_images_resized), -1)
test_images_flat = test_images_resized.reshape(len(test_images_resized), -1)

# Step 3: Apply PCA to reduce dimensions to 256
pca = PCA(n_components=256, random_state=42)
train_images_pca = pca.fit_transform(train_images_flat)
test_images_pca = pca.transform(test_images_flat)

# Step 4: Handle class imbalance by calculating class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_dict = dict(enumerate(class_weights))

# Step 5: Use TPOTClassifier for AutoML
tpot = TPOTClassifier(
    verbosity=2,
    generations=20,              # Reduced generations for quicker experiments
    population_size=30,          # Reduced population size to speed up processing
    random_state=42,
    scoring='balanced_accuracy', # Use balanced accuracy as the scoring metric due to imbalance
    cv=5,                        # 5-fold cross-validation for generalization
    n_jobs=-1                    # Use all CPU cores to speed up training
)

# Step 6: Train the model with reduced dimensions (PCA-transformed data)
tpot.fit(train_images_pca, train_labels)

# Step 7: Evaluate the model
score = tpot.score(test_images_pca, test_labels)
print(f"Balanced Accuracy: {score:.4f}")

# Step 8: Export the best pipeline
tpot.export('best_pipeline.py')


Optimization Progress:   0%|          | 0/630 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8083717357910907

Generation 2 - Current best internal CV score: 0.8132104454685098

Generation 3 - Current best internal CV score: 0.8228878648233486
